# Dependencias

In [1]:
import numpy as np 
import pandas as pd
import nltk

# Load Data

In [2]:
data = pd.read_csv('./mails_clean.csv')

In [3]:
print(data.shape)
data.head()

(42638, 10)


,Unnamed: 0,file,Message-ID,Date,From,To,Subject,X-cc,X-bcc,body
0,0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,{'phillip.allen@enron.com'},{'tim.belden@enron.com'},NaN,NaN,NaN,Here is our forecast
1,1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,{'phillip.allen@enron.com'},{'john.lavorato@enron.com'},Re:,NaN,NaN,Traveling to have a business meeting takes the...
2,2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,{'phillip.allen@enron.com'},{'leah.arsdall@enron.com'},Re: test,NaN,NaN,test successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,{'phillip.allen@enron.com'},{'randall.gay@enron.com'},NaN,NaN,NaN,"Randy, Can you send me a schedule of the sal..."
4,4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,{'phillip.allen@enron.com'},{'greg.piper@enron.com'},Re: Hello,NaN,NaN,Let's shoot for Tuesday at 11:45.


In [4]:
#data=data.set_index('Message-ID')
data=data.drop(columns='Unnamed: 0')
data=data.dropna(axis=0,subset=['body'],how='any')
print(data.shape)
data.head()

(42454, 9)


,file,Message-ID,Date,From,To,Subject,X-cc,X-bcc,body
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,{'phillip.allen@enron.com'},{'tim.belden@enron.com'},NaN,NaN,NaN,Here is our forecast
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,{'phillip.allen@enron.com'},{'john.lavorato@enron.com'},Re:,NaN,NaN,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,{'phillip.allen@enron.com'},{'leah.arsdall@enron.com'},Re: test,NaN,NaN,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,{'phillip.allen@enron.com'},{'randall.gay@enron.com'},NaN,NaN,NaN,"Randy, Can you send me a schedule of the sal..."
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,{'phillip.allen@enron.com'},{'greg.piper@enron.com'},Re: Hello,NaN,NaN,Let's shoot for Tuesday at 11:45.


# Pre-Procesamiento

In [5]:
import re
def clean_from_mails(message):
  mail_addresses = re.findall(r'\(?[A-Za-z0-9_\-\.]+@[A-Za-zb0-9_\-\.]+\.com\)?',message)
  if mail_addresses != []:
    for mail_address in mail_addresses:
      message=message.replace(mail_address,'').strip()
  return message

In [6]:
data['body_clean']=data['body'].map(clean_from_mails)

In [7]:
# Eliminar palabras con digitos y signos de puntuacion
data['body_clean']=data['body_clean'].map(lambda x: ' '.join(re.sub(r'[^a-zA-Z]',' ',x).split()))

In [8]:
# Pasar todo a minusculas
data['body_clean']=data['body_clean'].map(lambda x: x.lower())

## Stopwords

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kref/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
my_stopwords = stopwords.words('english')
# Excluir palabras de una sola letra
for i in range(ord('a'),ord('z')+1):
  my_stopwords.append(chr(i))
# Añadir would
my_stopwords = my_stopwords + ['also','please','would']
print(f'Stopwords final ({len(my_stopwords)}):')
print(my_stopwords)

Stopwords final (208):
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 

In [11]:
def clean_msg_stopwords(message_body):
    words = message_body.split()
    words = [w for w in words if w not in set(my_stopwords)]
    return ' '.join(words)

In [12]:
data['body_clean']=data.body_clean.apply(clean_msg_stopwords)

In [13]:
data=data.dropna(axis=0,subset=['body_clean'],how='any')

## Tokenizacion

In [14]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/kref/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
word_vector = []
for doc in data['body_clean']:
  word_vector.append(doc.split())
my_vocabulary = [item for sublist in word_vector for item in sublist]

In [16]:
word_vector[2]

['test', 'successful', 'way', 'go']

In [17]:
print(f'El vocabulario consta de {len(my_vocabulary)} elementos y {len(set(my_vocabulary))} palabras diferentes.')

El vocabulario consta de 4047756 elementos y 44475 palabras diferentes.


## Lematizacion

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/kref/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

def lemmatize_body(message_body):
  return ' '.join(lemma.lemmatize(word) for word in message_body.split())

In [20]:
lemma_vector = []
for doc in data['body_clean']:
  lemma_vector.append(lemmatize_body(doc).split())
my_lema_voc = [item for sublist in lemma_vector for item in sublist]

In [21]:
print(f'El vocabulario de lemas consta de {len(my_lema_voc)} elementos y {len(set(my_lema_voc))} palabras diferentes.')

El vocabulario de lemas consta de 4047756 elementos y 40903 palabras diferentes.


In [22]:
data['body_lemma'] = data['body_clean'].map(lemmatize_body)

In [23]:
data['body_lemma'][2]

'test successful way go'

In [24]:
type(data['body_lemma'])

pandas.core.series.Series

# Vectorizacion

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
word_vector = []
for doc in data['body_lemma']:
  word_vector.append(doc.split())
my_vocabulary = [item for sublist in word_vector for item in sublist]

In [27]:
word_vector[4]

['let', 'shoot', 'tuesday']

# Similaridad de Documentos

In [28]:
import gensim

In [29]:
dictionary = gensim.corpora.Dictionary(word_vector)

In [30]:
dictionary[0]

'forecast'

In [31]:
print(dictionary.token2id["forecast"])

0


In [32]:
print("Number of words in dictionary:",len(dictionary))

Number of words in dictionary: 40903


In [33]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in word_vector]

In [35]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=42454, num_nnz=2761697)


## Procesamiento Similaridad

In [36]:
sims = gensim.similarities.Similarity('./',tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

/home/kref/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Similarity index with 42454 documents in 1 shards (stored under ./)
<class 'gensim.similarities.docsim.Similarity'>


## Query

In [58]:
query_doc = [w.lower() for w in word_tokenize("duplication of efforts")]

In [59]:
query_doc_bow = dictionary.doc2bow(query_doc)

In [60]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [61]:
sims[query_doc_tf_idf]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [62]:
sim_w_index = list(enumerate(sims[query_doc_tf_idf]))

In [63]:
x = list(reversed(sorted(sim_w_index,key=lambda x: x[1])))

In [64]:
x[0:10]

[(41927, 0.22135752),
 (41732, 0.22135752),
 (7349, 0.21695684),
 (5554, 0.21695684),
 (3848, 0.21695684),
 (30251, 0.18691298),
 (35294, 0.17971303),
 (35402, 0.1345826),
 (26615, 0.12446223),
 (25279, 0.12446223)]

In [65]:
[x[0][0]][0]

41927

In [66]:
data['body'][x[0][0]]

'Is it really necessary to coordinate everything through ENW?  I am not sure  people will do that?  I think ENW should be informed but not be in control of  the registration.  Have the ENW people requested that they be in charge of  this process?  Will they care if we change it?     Leslie Hansen  12/12/2000 02:14 PM        To: Mark E Haedicke/HOU/ECT@ECT    cc:     Subject: Enron Policy re Use of Third Party Trading Platforms  Mark:  I wanted to follow up to see if you have had an opportunity to consider the  proposed policy attached below.  Currently, the traders have no guidelines  governing their use of third party electronic trading platforms, and there  has been no centralized system for monitoring such use.    Please give me a call with any questions or comments.  Leslie  Enron Policy re Use of Third Party Trading Platforms  Mark:  Mark Taylor and I have worked with Andy Zipper to develop a policy relating  to use of third party electronic trading platforms.  We propose that you